In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_data = pd.read_excel('Data_Train.xlsx')
train_data.shape

(10683, 11)

In [3]:
test_data = pd.read_excel('Test_Set.xlsx')
test_data.shape

(2671, 10)

In [4]:
test_data.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
dtype: int64

In [5]:
train_data.loc[train_data.Route.isnull(),'Route'] = 'DEL → MAA → COK'
train_data.loc[train_data.Total_Stops.isnull(),'Total_Stops'] = '1 stop'

In [6]:
def get_date(str):
    return str.split("/")[0]

In [7]:
train_data["Day_of_journey"] = train_data.Date_of_Journey.apply(get_date)
test_data["Day_of_journey"] = test_data.Date_of_Journey.apply(get_date)

In [8]:
def get_month(str):
    return str.split("/")[1]

In [9]:
train_data["Month_of_journey"] = train_data.Date_of_Journey.apply(get_month)
test_data["Month_of_journey"] = test_data.Date_of_Journey.apply(get_month)

In [10]:
train_data.drop("Date_of_Journey", axis =1, inplace = True)
test_data.drop("Date_of_Journey", axis =1, inplace = True)

In [11]:
train_data.Total_Stops.replace('non-stop','0 stop', inplace = True)
test_data.Total_Stops.replace('non-stop','0 stop', inplace = True)

train_data.Total_Stops = train_data.Total_Stops.str.split().str.get(0)
test_data.Total_Stops = test_data.Total_Stops.str.split().str.get(0)

In [12]:
def convert_duration(str):
    duration_array = str.split()
    hour = duration_array[0][:-1]
    min = '0'
    if(len(duration_array)>1):
        min = duration_array[1][:-1]
    
    return int(hour)*60 + int(min)

In [13]:
train_data.Duration = train_data.Duration.apply(convert_duration)
test_data.Duration = test_data.Duration.apply(convert_duration)

In [14]:
train_data.drop('Arrival_Time', axis =1, inplace=True)
test_data.drop('Arrival_Time', axis =1, inplace=True)

In [15]:
def update_route(str):
    route_array = str.split("→")
    val = ''
    for x in route_array[1:-1]:
        val = val + " " + x
    return val

In [16]:
train_data.Route = train_data.Route.apply(update_route)
test_data.Route = test_data.Route.apply(update_route)

In [17]:
train_data['stop 1'] = train_data['Route'].str.split().str.get(0)
train_data['stop 2'] = train_data['Route'].str.split().str.get(1)
train_data['stop 3'] = train_data['Route'].str.split().str.get(2)
train_data['stop 4'] = train_data['Route'].str.split().str.get(3)

test_data['stop 1'] = test_data['Route'].str.split().str.get(0)
test_data['stop 2'] = test_data['Route'].str.split().str.get(1)
test_data['stop 3'] = test_data['Route'].str.split().str.get(2)
test_data['stop 4'] = test_data['Route'].str.split().str.get(3)

train_data.drop('Route', axis = 1, inplace=True)
test_data.drop('Route', axis = 1, inplace=True)

In [18]:
def convert_dep_time(str):
    dep_time_array = str.split(':')
    return int(dep_time_array[0])*60 + int(dep_time_array[1])

In [19]:
train_data.Dep_Time = train_data.Dep_Time.apply(convert_dep_time)
test_data.Dep_Time = test_data.Dep_Time.apply(convert_dep_time)

In [20]:
train_data.fillna('N.A.', axis =1, inplace=True)
test_data.fillna('N.A.', axis =1, inplace=True)

In [21]:
train_data.shape

(10683, 14)

In [22]:
test_data.shape

(2671, 13)

In [23]:
train_objs_num = len(train_data)
dataset = pd.concat(objs=[train, test], axis=0)
dataset = pd.get_dummies(dataset)
train = copy.copy(dataset[:train_objs_num])
test = copy.copy(dataset[train_objs_num:])

NameError: name 'train' is not defined

In [ ]:
len_train = len(train_data)
len_train

In [ ]:
data = pd.concat([train_data,test_data], axis = 0)

In [ ]:
data.shape

In [ ]:
data_new = pd.get_dummies(data, columns=['Airline','Source','Destination','Additional_Info','Day_of_journey',
                                         'Month_of_journey','stop 1','stop 2','stop 3','stop 4'])

In [ ]:
data_new.shape

In [ ]:
train_data_new = data_new[:len_train]
test_data_new = data_new[len_train:]

In [ ]:
train_data_new.shape

In [ ]:
test_data_new.shape

In [ ]:
test_data_new.drop('Price', axis =1, inplace = True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
x = train_data_new.drop('Price', axis =1)
y = train_data_new.Price

In [ ]:
#Lasso & Ridge

In [44]:
x_sc = pd.DataFrame(MinMaxScaler().fit_transform(x), columns= x.columns)

In [49]:
#Perform XGBoost

import xgboost as xgb

xgb_m = xgb.XGBRegressor()

In [58]:
from sklearn.model_selection import RandomizedSearchCV

In [59]:
parameters = {"n_estimators":[1000,2000], 
        "max_depth": [3,6,10],
        "min_child_weight":[3,5],
        "gamma":[0.1,0.5,1],
        'subsample':[0.5,0.7,0.9],
        "learning_rate":[0.01,0.05,0.1,0.3]
       }

In [60]:
rs = RandomizedSearchCV(xgb_m, param_distributions=parameters,n_iter=10)

In [61]:
rs.fit(x_sc,y)

[11:08:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:11:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:15:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:18:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:19:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:20:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:20:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:22:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:23:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
          estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'n_estimators': [1000, 2000], 'max_depth': [3, 6, 10], 'min_child_weight': [3, 5], 'gamma': [0.1, 0.5, 1], 'subsample': [0.5, 0.7, 0.9], 'learning_rate': [0.01, 0.05, 0.1, 0.3]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [62]:
rs.best_params_

{'subsample': 0.9,
 'n_estimators': 1000,
 'min_child_weight': 3,
 'max_depth': 6,
 'learning_rate': 0.1,
 'gamma': 0.1}

In [63]:
rs.best_score_

0.9040303144902058

In [65]:
from sklearn.model_selection import cross_val_score
xgb_m = xgb.XGBRegressor(n_estimators=1000, subsample=0.9, min_child_weight=3, max_depth=6,learning_rate=0.1,gamma=0.1,objective='reg:squarederror')
cross_val_score(xgb_m,x_sc,y,cv=10).mean()

0.9100191949646186

In [66]:
xgb_m.fit(x_sc,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0.1,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=3, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=0.9, verbosity=1)

In [67]:
test_data_sc = pd.DataFrame(MinMaxScaler().fit_transform(test_data_new), columns= test_data_new.columns)

In [68]:
price_df = pd.DataFrame(xgb_m.predict(test_data_sc), columns=['Price'])

In [69]:
price_df.to_excel("sample.xlsx")